# Corpus File Preprocessing for Word2Vec:
## 1. Load and read the corpus.
## 2. Tokenize the text.
## 3. Lowercase all words.
## 4. Remove stop words and non-alphabetic words.
## 5. (Optional) Lemmatize words.
## 6. Optionally, subsample frequent words.
## 7. Prepare the corpus for Word2Vec (list of tokenized sentences).
## 8. Train the Word2Vec model using Gensim or other frameworks.

# PARTE 1: Costruzione del Modello Word2Vec

### Import delle librerie

In [ ]:
import gensim
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
#nltk.download('stopwords')
#nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import string

import spacy

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt



from sklearn.cluster import KMeans

from sklearn.feature_extraction.text import TfidfVectorizer

import os
import pandas as pd
import numpy as np

from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

### Lettura e tokenizzazione per frasi del Corpus documentale

In [ ]:
with open(r"/home/acarugat/MyRepo1/Testi/Manzoni/Corpus.txt", "r", encoding="utf-8") as file:
    corpus = file.read()
corpus=sent_tokenize(corpus, language='italian')

### Tokenizzazione per parole del Corpus documentale togliendo la punteggiatura ... e non solo

In [ ]:
tokenized_corpus = [word_tokenize(sentence.lower(), language='italian') for sentence in corpus]

punteggiatura = string.punctuation+"«»’"

tokenized_corpus2=[]
for sentence in tokenized_corpus:
    sentence2=[]
    for word in sentence:
        if word not in punteggiatura:
            sentence2.append(word)
    tokenized_corpus2.append(sentence2)
    
tokenized_corpus=tokenized_corpus2   
    
#okenized_corpus = [word for word in tokenized_corpus if word not in string.punctuation]

### Togliamo gli apostrofi

In [ ]:
tokenized_corpus_noap = []
for sottolista in tokenized_corpus:
    nuova_sottolista = []
    for elemento in sottolista:
        if "'" in elemento:  # Se l'elemento contiene un apostrofo
            nuova_sottolista.extend(elemento.split("'"))  # Splitta e aggiungi le parti
        else:
            nuova_sottolista.append(elemento)  # Aggiungi direttamente l'elemento
    tokenized_corpus_noap.append(nuova_sottolista)

### Togliamo le stopwords ... e non solo

In [ ]:
stop_words = set(stopwords.words('italian'))
stop_words.update(["d", "gl", "de", "et", "s", "o", "..."])
cleaned_corpus = [[word for word in sentence if word not in stop_words] for sentence in tokenized_corpus_noap]

### Salviamo il Corpus documentale preprocessato

In [ ]:
with open(r"/home/acarugat/MyRepo1/Testi/Manzoni/Corpus-PP.txt", "w", encoding="utf-8") as file:
    for sentence in cleaned_corpus:
        file.write(" ".join(sentence) + "\n")

### Generiamo il Modello Word2Vec a partire dal Corpus documentale preprocessato

In [ ]:
# Train the Word2Vec model
model = Word2Vec(corpus_file=r"/home/acarugat/MyRepo1/Testi/Manzoni/Corpus-PP.txt", vector_size=100, window=5, min_count=5, workers=4)

# Save the trained model
model.save("word2vec_model.model")

### Valutiamo il Modello ottenuto

In [ ]:
print (model.wv.get_vecattr("lago", "count"))
print (model.wv.most_similar("lago"))
print (model.wv.similarity("lago", "fiume"))

### Addestriamo il Modello Word2Vec model con più iterazioni

In [ ]:
model = Word2Vec(corpus_file=r"/home/acarugat/MyRepo1/Testi/Manzoni/Corpus-PP.txt", vector_size=100, window=5, min_count=5, workers=4, epochs=100, sg=0)

### Valutiamo il Modello ottenuto

In [ ]:
print (model.wv.get_vecattr("lago", "count"))
print (model.wv.most_similar("lago"))
print (model.wv.similarity("lago", "fiume"))

### Addestriamo il Modello Word2Vec model con più iterazioni e algoritmo "skip-grammar"

In [ ]:
model = Word2Vec(corpus_file=r"/home/acarugat/MyRepo1/Testi/Manzoni/Corpus-PP.txt", vector_size=100, window=5, min_count=5, workers=4, epochs=100, sg=1)

### Valutiamo il Modello ottenuto

In [ ]:
print (model.wv.get_vecattr("lago", "count"))
print (model.wv.most_similar("lago"))
print (model.wv.similarity("lago", "fiume"))

# PARTE 1: Analisi della Frequenza delle Parole

### Otteniamo la lista delle parole presenti nel Corpus

In [ ]:
from itertools import chain

# Flatten the list
flattened_corpus = list(chain.from_iterable(cleaned_corpus))

# Get unique tokens
parole_presenti = list(set(flattened_corpus))


### Otteniamo la lista delle parole presenti nel Modello (teniamo presente che min_count = 5)

In [ ]:
#In Gensim's Word2Vec, the vocabulary already contains unique words
tokens_unici= list(model.wv.index_to_key)

## Per ogni parola presente nel Modello otteniamo la frequenza

In [ ]:
    tabella = []
    for parola in tokens_unici:
        c=model.wv.get_vecattr(parola,"count")
        tabella.append({"Parola":parola, "Frequenza":c})
    df=pd.DataFrame(tabella)

### Stampiamo il grafico delle frequenze delle parole con nltk.FreqDist

In [ ]:
freq_dist=nltk.FreqDist(flattened_corpus)
freq_dist.plot(10, cumulative=False)

### Stampiamo il grafico delle frequenze delle parole con seaborn

In [ ]:
# Imposta lo stile di Seaborn (opzionale)
# sns.set(style="whitegrid")

# Crea il grafico a barre
plt.figure(figsize=(10, 6))
sns.barplot(x='Frequenza', y='Parola', data=df.sort_values('Frequenza', ascending=False).head(10))

# Aggiungi titolo e etichette agli assi
plt.title('Le 10 parole più frequenti', fontsize=16)
plt.xlabel('Frequenza', fontsize=12)
plt.ylabel('Parola', fontsize=12)

# Mostra il grafico
plt.show()

# PARTE 3: CLUSTERIZZAZIONE

### Generazione dei Clusters

In [ ]:
# Esempio: vettori = [model[word] for word in parole_presenti]
vettori = [model.wv[word] for word in tokens_unici]
           
# Converti la lista in un array NumPy
vettori_array = np.array(vettori)

# Applica t-SNE
# t-SNE (t-distributed Stochastic Neighbor Embedding) 
# è una tecnica di riduzione della dimensione utilizzata per visualizzare dati ad alta dimensione 
# in uno spazio a bassa dimensione (tipicamente 2D o 3D)
tsne = TSNE(n_components=2, random_state=42)
# Generiamo un "modello" ridotto  a 2 dimensioni
vettori_ridotti = tsne.fit_transform(vettori_array)

# Visualizza i risultati
#plt.scatter(vettori_ridotti[:, 0], vettori_ridotti[:, 1])
#for i, parola in enumerate(tokens_unici):
#    plt.text(vettori_ridotti[i, 0] + 0.01, vettori_ridotti[i, 1] + 0.01, parola, fontsize=9)
#plt.title("Visualizzazione t-SNE")
#plt.show()


# Imposta il numero di cluster
num_clusters = 100

# Esegui il clustering
#kmeans = KMeans(n_clusters=num_clusters, n_init='auto', random_state=42)
'''
La funzione KMeans è un algoritmo di clustering (o raggruppamento) non supervisionato 
che viene utilizzato per suddividere un dataset in un numero specificato di gruppi o cluster. 
Ogni gruppo rappresenta un insieme di punti dati che sono "simili" tra loro, 
basandosi su una misura di distanza (tipicamente la distanza euclidea).
'''
#kmeans = KMeans(n_clusters=num_clusters, init='k-means++', n_init='auto', max_iter=300, tol=0.0001, verbose=0, random_state=None, copy_x=True, algorithm='lloyd')
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', n_init=100, max_iter=300, tol=0.0001, verbose=0, random_state=None, copy_x=True, algorithm='lloyd')

kmeans.fit(vettori)

# Ottieni i cluster
'''
 l'attributo labels_ contiene un array di etichette, una per ogni punto nel dataset di input, 
 che indica a quale cluster ogni punto è stato assegnato.
'''
clusters = kmeans.labels_

# Raggruppa le parole per cluster
cluster_parole = {i: [] for i in range(num_clusters)}
for parola, cluster in zip(tokens_unici, clusters):
    cluster_parole[cluster].append(parola)

# Visualizza i cluster
for cluster, parole in cluster_parole.items():
    print(f"Cluster {cluster}: {parole}")
    plt.figure(figsize=(10, 10))
    for i, parola in enumerate(parole):
        plt.scatter(vettori_ridotti[i, 0], vettori_ridotti[i, 1])
        plt.annotate(parola, (vettori_ridotti[i, 0], vettori_ridotti[i, 1]))
    plt.show()


# PARTE 4: Analisi delle similarità lessicali

### Similarità tra due documenti tramite sklearn.feature_extraction.text.TfidfVectorizer

In [ ]:
'''
La matrice di similarità

La matrice risultante da tfidf * tfidf.T è una matrice di dimensioni 2x2 
(poiché stiamo confrontando solo due documenti, a e b). 
Il prodotto scalare tra i due vettori TF-IDF è calcolato per ciascun paio di documenti. 
La matrice risultante avrà la seguente struttura:
Matrice di similaritaˋ=(sim(a,a)sim(a,b)sim(b,a)sim(b,b))
Matrice di similaritaˋ=(sim(a,a)sim(b,a)​sim(a,b)sim(b,b)​)

Dove:

    sim(a, a) è la similarità coseno tra il documento a e se stesso. 
    Questo valore sarà sempre 1, perché ogni vettore è perfettamente simile a se stesso.
    sim(b, b) è la similarità coseno tra il documento b e se stesso. 
    Anche questo valore sarà 1.
    sim(a, b) è la similarità coseno tra i due documenti, a e b.
    sim(b, a) è la stessa cosa di sim(a, b), poiché la similarità coseno è simmetrica.
'''
vectorizer = TfidfVectorizer()

def calcola_similarita(a, b): 
    tfidf = vectorizer.fit_transform([a, b])
    #print ("Colonna 0, Riga 0:", ((tfidf * tfidf.T).toarray())[0,0])
    #print ("Colonna 0, Riga 1:", ((tfidf * tfidf.T).toarray())[0,1])
    #print ("Colonna 1, Riga 0:", ((tfidf * tfidf.T).toarray())[1,0])
    #print ("Colonna 1, Riga 1:", ((tfidf * tfidf.T).toarray())[1,1])
    return ((tfidf * tfidf.T).toarray())[0,1]

file1_path = r"/home/acarugat/MyRepo1/Testi/Manzoni/Fermo-e-Lucia.txt"

with open(file1_path, 'r', encoding='utf-8') as file: 
    testo1 = file.read()

file2_path = r"/home/acarugat/MyRepo1/Testi/Manzoni/PromessiSposi-1840.txt"

with open(file2_path, 'r', encoding='utf-8') as file: 
    testo2 = file.read()

s=calcola_similarita(testo1,testo2)

print (s)


### Similarità tra insiemi di documenti: Per ogni file presente in una directory specificata in input,
### metti in un dataframe: il nome del file e il contenuto (testo) del documento

In [ ]:
# Initialize lists to store filenames and contents
filenames = []
contents = []

# Walk through the home directory and read all text files
print ("Dimmi che documenti vuoi analizzare: Manzoni Presidenti Giornali o Parlamento")
line = input()

for root, dirs, files in os.walk(r"/home/acarugat/MyRepo1/Testi/"+line):
    for file in files:
        if file not in ["Corpus.txt", "Corpus-PP.txt", "PromessiSposi-stopword.txt", "MattarellaFine2024-preprocessato.txt"]:
            if file.endswith(".txt"):  # Check if the file is a text file
                filepath = os.path.join(root, file)
                try:
                    with open(filepath, 'r', encoding='utf-8') as f:
                        filenames.append(filepath)  # Store the file path
                        contents.append(f.read())  # Store the file content
                except Exception as e:
                    print(f"Error reading {filepath}: {e}")

# Create the dataset (DataFrame)
dataset = pd.DataFrame({
    "filename": filenames,
    "content": contents
})

# Display the dataset
# print(dataset)


### creiamo una matrice M per contenere le similarità di testo_i con testo_j usando tqdm

In [ ]:
'''
tqdm è una libreria Python che fornisce una barra di progresso 
facile da usare e visivamente attraente per monitorare il progresso di loop, operazioni 
o processi che richiedono tempo. 
Il nome tqdm sta per "taqaddum", che significa "progresso" in arabo.
'''

M = np.zeros((dataset.shape[0], dataset.shape[0])) 
#N.B.: essendo la matrice M quadrata, dataset.shape[0] = numero di righe = dataset.shape[1] = numero di colonne 
for i, row in tqdm(dataset.iterrows(), total=dataset.shape[0], desc='1st level'): # definiamo i
    for j, next_row in dataset.iterrows(): # definiamo j
        M[i, j] = calcola_similarita(row.content, next_row.content) # popoliamo la matrice con i risultati

### Creiamo un DataFrame contenente le similarità

In [ ]:
labels=dataset.filename.str.split('/').str[5:].str[1]
labels=labels.str.split('.').str[0]
similarity_df = pd.DataFrame(M, columns=labels, index=labels) # creiamo un dataframe

In [ ]:
### Rappresentazione grafica mediante heatmap delle similarità tra diversi documenti

In [ ]:
# creiamo la visualizzazione
plt.figure(figsize=(12, 12))
sns.heatmap(
			similarity_df,
			square=True, 
			annot=True, 
			robust=True,
			fmt='.2f',
			annot_kws={'size': 7, 'fontweight': 'bold'},
			yticklabels=similarity_df.columns,
			xticklabels=similarity_df.columns,
			cmap="YlGnBu",
            #mask=mask
			mask=None
)

plt.title('Heatmap delle similarità tra testi', fontdict={'fontsize': 24})
plt.show()

# PARTE 5: Che Parole Usi

### Carichiamo Modello Italiano di Spacy

In [ ]:
# Carica il modello di lingua italiana di spaCy
nlp = spacy.load("it_core_news_sm")

### Definiamo funzione "trova_lessemi"

In [ ]:
nlp.max_length = 10000000
# Funzione per estrarre i lessemi (lemmi) da un testo, escludendo le stop words
def trova_lessemi_da_file(nome_file):
    try:
        # Leggi il contenuto del file
        with open(nome_file, "r", encoding="utf-8") as file:
            testo = file.read()
        
        # Analizza il testo
        doc = nlp(testo)
        
        # Estrai i lessemi, escludendo punteggiatura, spazi e stop words
        lessemi = [
            token.lemma_
            for token in doc
            if not token.is_punct and not token.is_space and not token.is_stop and not token.pos_ == "DET" and not token.pos_ == "ADP"
        ]
        return lessemi
    except FileNotFoundError:
        print(f"Errore: Il file '{nome_file}' non è stato trovato.")
        return []


### Costruiamo Lessico di Base da File di Input contenente Lista di Parole Italiane

In [ ]:
with open("/home/acarugat/paroleitaliane.txt", "r", encoding="utf-8") as file:
    lessicobase = file.read().lower()

### Definiamo Funzione che Rimuove Accenti

In [ ]:
import unicodedata

# Funzione per rimuovere gli accenti
def rimuovi_accenti(testo):
    testo_normalizzato = unicodedata.normalize("NFD", testo)
    testo_senza_accenti = "".join(char for char in testo_normalizzato if not unicodedata.combining(char))
    return testo_senza_accenti

### Confrontiamo il Lessico del Documento da analizzare con il Lessico di Base

In [ ]:
parole_presenti = []
parole_assenti = []
# Specifica il nome del file di input
nome_file = input()
with open(nome_file, "r", encoding="utf-8") as file:
    testo = file.read().lower()
    
    # Passa il testo alla pipeline di spaCy
doc = nlp(testo)

# Esegui alcune analisi
for parola in doc:
    if not parola.is_punct and not parola.is_space and not parola.is_stop and not parola.pos_ == "DET" and not parola.pos_ == "ADP":
        print(f"Token: {parola.text}, POS: {parola.pos_}, Lemma: {parola.lemma_}")
        lemma_senza_accenti = rimuovi_accenti (parola.lemma_)
        lemma_senza_accenti_minuscolo = lemma_senza_accenti.lower()
        if lemma_senza_accenti_minuscolo in lessicobase:
            parole_presenti.append(parola.text)
        else:
            parole_assenti.append(parola.text)

### Stampiamo Risultati: Parole Presenti

In [ ]:
parole_presenti=(set(parole_presenti))
print (parole_presenti)

### Stampiamo Risultati: Parole Assenti

In [ ]:
parole_assenti=(set(parole_assenti))
print (parole_assenti)